# Distributed workflows in programs

In this document we will learn how to run distributed workflows inside a program.

Let's create another file with our updated program [./source_files/program_4.py](./source_files/program_4.py). 

First let's see how file would look like and then we will discuss what is happening there

```python
# source_files/program_4.py

from quantum_serverless import get_arguments, save_result, distribute_task, get

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler


@distribute_task()
def distributed_sample(circuit: QuantumCircuit):
    """Distributed task that returns quasi distribution for given circuit."""
    return Sampler().run(circuit).result().quasi_dists[0]


arguments = get_arguments()
circuits = arguments.get("circuits")


# run distributed tasks as async function
# we get task references as a return type
sample_task_references = [
    distributed_sample(circuit)
    for circuit in circuits
]

# now we need to collect results from task references
results = get(sample_task_references)

save_result({
    "results": results
})
```

A lot of new things are happening in this program.

Let's begin with `distribute_task` decorator. `distribute_task` when applied to function converts is to distributed task, which means that this function will be executed on compute resources asyncronously and in parallel to main context of a program. 

When you call converted function it will return reference to function execution instead of result. 
In order to get results back we need to call `get` on function reference. `get` will wait until function is finished and return result of funciton execution. 

Essentially what we are doing in this program:
- reading circuit from aruments
- calling `distributed_sample` function for each of the circuit
- now each of `distributed_sample` funciton is running in parallel
- we get results of all executions by calling `get`
- saving results


In [1]:
from quantum_serverless import QuantumServerless, GatewayProvider

In [2]:
provider = GatewayProvider(
    username="user",
    password="password123",
    host="http://localhost:8000",
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

Let's create list of random circuit which we will be using as arguments

In [12]:
from qiskit.circuit.random import random_circuit

circuits = [random_circuit(2, 2) for _ in range(5)]
[circuit.measure_all() for circuit in circuits]
circuits

Run program as usual

In [13]:
from quantum_serverless import Program

program = Program(
    title="Program with distributed workflow",
    entrypoint="program_4.py",
    working_dir="./source_files/"
)

job = serverless.run_program(program, arguments={"circuits": circuits})
job

<Job | 6bd54632-bce0-4214-a5e1-04f931ab7dcb>

In [18]:
job.status()

'SUCCEEDED'

In [20]:
job.result()

'{"results": [{"0": 0.0019388099833208, "1": 0.0246690007437, "2": 0.0709273872101196, "3": 0.9024648020628592}, {"0": 0.4999999999999999, "2": 0.4999999999999999}, {"0": 0.0008967561725936, "1": 2.42854928976e-05, "3": 0.9990789583345085}, {"0": 0.0769441987062216, "1": 0.0769441987062216, "2": 0.4230558012937782, "3": 0.4230558012937782}, {"0": 0.1019745637494164, "1": 0.2181326517505121, "2": 0.216589213671712, "3": 0.4633035708283594}]}'